In [75]:
import pandas as pd
import numpy as np
from geopy.distance import vincenty
import fiona
from shapely.geometry import shape, Polygon, MultiPolygon, Point,\
    MultiLineString, MultiPoint
import cPickle as pickle

####Only want to add street features for the rows not already done, 7500 - 10999

In [2]:
df = pd.read_pickle('df_1to7499_cleaned.pkl')

In [3]:
df2 = pd.read_pickle('df_1to10999_cleaned.pkl')

In [4]:
df2.shape

(7919, 12)

In [5]:
df2.tail(25)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address
10962,10963,159493,1211 MINOR AVE ...,MINOR AVE BETWEEN SENECA ST AND UNIVERSITY ST ...,2012-07-31 07:00:00,2012-08-03 07:00:00,2012-08-03 07:00:00,3 days 00:00:00,3,47.609386,-122.332198,"University St, Seattle, WA 98101, USA"
10963,10964,163278,SW Roxbury St and 47th Ave SW intersection the...,S ROXBURY ST BETWEEN MARCUS AVE S AND 48TH AVE...,2012-09-02 07:00:00,2012-09-05 07:00:00,2012-09-05 07:00:00,3 days 00:00:00,3,47.505143,-122.273783,"48th Ave S, Seattle, WA, USA"
10964,10965,162384,...,W RAYE ST BETWEEN DEAD END 3 AND 23RD AVE W ...,2012-08-24 07:00:00,2012-08-29 07:00:00,2012-08-29 07:00:00,5 days 00:00:00,5,47.645996,-122.385913,"23rd Ave W, Seattle, WA, USA"
10971,10972,160024,"8th Ave. NW., betweeen NW 65th and Market sts ...",8TH AVE NW AND NW 65TH ST ...,2012-08-05 07:00:00,2012-08-07 07:00:00,2012-08-07 07:00:00,2 days 00:00:00,2,47.675980,-122.366063,"8th Ave NW & NW 65th St, Seattle, WA 98117, USA"
10973,10974,165468,2415 NE 80th St crosswalk ...,NE 80TH ST BETWEEN 24TH AVE NE AND 25TH AVE NE...,2012-09-18 07:00:00,2012-09-19 07:00:00,2012-09-19 07:00:00,1 days 00:00:00,1,47.717509,-122.301791,"25th Ave NE, Seattle, WA, USA"
10975,10976,111842,ROOSEVELT WY NE BTW 65TH AVE NE & 75TH AVE NE ...,ROOSEVELT WAY NE BETWEEN NE 65TH ST AND NE 66T...,2012-04-06 07:00:00,2012-04-11 07:00:00,2012-04-23 07:00:00,17 days 00:00:00,17,47.676580,-122.315665,"NE 66th St, Seattle, WA 98115, USA"
10976,10977,155406,NE 52ND AT 5TH AVE ...,5TH AVE NE BETWEEN NE 51ST ST AND NE 52ND ST ...,2012-07-03 07:00:00,2012-07-11 07:00:00,2012-07-11 07:00:00,8 days 00:00:00,8,47.666443,-122.303444,"NE 52nd St, Seattle, WA 98105, USA"
10978,10979,113373,328 M L KING JR WAY E ...,M L KING JR WAY BETWEEN E ALDER ST AND E JEFFE...,2012-04-18 07:00:00,2012-04-19 07:00:00,2012-04-19 07:00:00,1 days 00:00:00,1,47.606171,-122.303803,"E Jefferson St, Seattle, WA, USA"
10979,10980,120408,5260 17TH AVE NE ...,17TH (SB) AVE NE BETWEEN NE 52ND ST AND NE 55T...,2012-06-02 07:00:00,2012-06-03 07:00:00,2012-06-03 07:00:00,1 days 00:00:00,1,47.668522,-122.295630,"NE 55th St, Seattle, WA 98105, USA"
10980,10981,113983,...,SENECA ST BETWEEN BOYLSTON AVE AND HARVARD AVE...,2012-04-23 07:00:00,2012-04-27 07:00:00,2012-04-27 07:00:00,4 days 00:00:00,4,47.615239,-122.322122,"Harvard Ave, Seattle, WA 98122, USA"


In [6]:
df.tail(2)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address
7498,7499,225583,NE 130TH ST BETWEEN LAKE CITY WAY NE AND 33RD ...,NE 130TH ST BETWEEN LAKE CITY WAY NE AND 33RD ...,2013-10-01 07:00:00,2013-10-01 07:00:00,2013-10-02 07:00:00,1 days,1,47.744614,-122.294066,"33rd Ave NE, Seattle, WA, USA"
7499,7500,210335,EB bike lane between 14th and 16th Ave ...,E UNION ST BETWEEN 15TH AVE AND 16TH AVE ...,2013-07-10 07:00:00,2013-07-12 07:00:00,2013-07-12 07:00:00,2 days,2,47.610378,-122.311513,"16th Ave, Seattle, WA 98122, USA"


In [7]:
df.shape

(5373, 12)

In [8]:
df.loc[7498:7503,:]

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address
7498,7499,225583,NE 130TH ST BETWEEN LAKE CITY WAY NE AND 33RD ...,NE 130TH ST BETWEEN LAKE CITY WAY NE AND 33RD ...,2013-10-01 07:00:00,2013-10-01 07:00:00,2013-10-02 07:00:00,1 days,1,47.744614,-122.294066,"33rd Ave NE, Seattle, WA, USA"
7499,7500,210335,EB bike lane between 14th and 16th Ave ...,E UNION ST BETWEEN 15TH AVE AND 16TH AVE ...,2013-07-10 07:00:00,2013-07-12 07:00:00,2013-07-12 07:00:00,2 days,2,47.610378,-122.311513,"16th Ave, Seattle, WA 98122, USA"


In [9]:
df3 = df2.loc[7500:,:]

In [10]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2546 entries, 7500 to 10999
Data columns (total 12 columns):
OBJECTID         2546 non-null int64
WOKEY            2546 non-null int64
LOCATION         2546 non-null object
ADDRDESC         2546 non-null object
INITDT_dt        2546 non-null datetime64[ns]
FLDSTARTDT_dt    2546 non-null datetime64[ns]
FLDENDDT_dt      2546 non-null datetime64[ns]
DURATION         2546 non-null timedelta64[ns]
DURATION_td      2546 non-null float64
latitude         2546 non-null float64
longitude        2546 non-null float64
address          2546 non-null object
dtypes: datetime64[ns](3), float64(3), int64(2), object(3), timedelta64[ns](1)
memory usage: 258.6+ KB


In [11]:
df3.head()

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address
7500,7501,259536,...,N 130TH ST BETWEEN MERIDIAN AVE N AND CORLISS ...,2014-02-27 08:00:00,2014-03-05 08:00:00,2014-03-05 08:00:00,6 days,6,47.710445,-122.331354,"Corliss Ave N, Seattle, WA, USA"
7502,7503,208191,1307 N Northgate Way Between Ashworth and Ston...,N NORTHGATE WAY BETWEEN ASHWORTH AVE N AND N 1...,2013-06-30 07:00:00,2013-07-02 07:00:00,2013-07-02 07:00:00,2 days,2,47.606209,-122.332071,"Seattle, WA, USA"
7503,7504,221221,NE 63rd St. between 16th and 17th NE. I ...,NE 63RD ST BETWEEN 16TH AVE NE AND 17TH AVE NE...,2013-09-03 07:00:00,2013-09-05 07:00:00,2013-09-05 07:00:00,2 days,2,47.699426,-122.309494,"17th Ave NE, Seattle, WA, USA"
7504,7505,192738,...,THOMAS ST BETWEEN MINOR AVE N AND PONTIUS AVE ...,2013-03-25 07:00:00,2013-03-28 07:00:00,2013-03-28 07:00:00,3 days,3,47.621689,-122.331715,"Pontius Ave N, Seattle, WA 98109, USA"
7505,7506,175763,4506 S Bond ...,S BOND ST BETWEEN DEAD END AND BEACON AVE S ...,2012-11-27 08:00:00,2012-11-29 08:00:00,2012-11-29 08:00:00,2 days,2,47.534119,-122.289440,"Beacon Ave S, Seattle, WA, USA"


In [12]:
df3.tail()

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address
10992,10993,154426,- \n\n \n\n \n\n 49TH AVE SW BETWEEN SW EDMU...,49TH AVE SW BETWEEN SW EDMUNDS ST AND SW HUDSO...,2012-06-25 07:00:00,2012-06-25 07:00:00,2012-06-26 07:00:00,1 days,1,47.557458,-122.387559,"SW Hudson St, Seattle, WA, USA"
10993,10994,160571,"800 FAIRVIEW AVE N, ...",FAIRVIEW AVE N BETWEEN FAIRVIEW PL N AND ALOHA...,2012-08-09 07:00:00,2012-08-14 07:00:00,2012-08-14 07:00:00,5 days,5,47.627149,-122.343223,"Aloha St, Seattle, WA, USA"
10994,10995,113899,"12TH AND BOREN AVE S, ...",12TH AVE S BETWEEN BOREN AVE S AND 12TH (TURN ...,2012-04-23 07:00:00,2012-04-25 07:00:00,2012-04-25 07:00:00,2 days,2,47.606209,-122.332071,"Seattle, WA, USA"
10998,10999,96785,9719 3rd Ave NW ...,3RD AVE NW BETWEEN NW 97TH ST AND NW 100TH ST ...,2011-12-28 08:00:00,2011-12-30 08:00:00,2011-12-30 08:00:00,2 days,2,47.701542,-122.374093,"NW 100th St, Seattle, WA 98177, USA"
10999,11000,107829,Rainier Ave S to S Massachusetts ...,RAINIER AVE S BETWEEN RAINIER NB (I90 EB ON RP...,2012-03-19 07:00:00,2012-03-20 07:00:00,2012-03-20 07:00:00,1 days,1,47.588483,-122.313814,"S Massachusetts St, Seattle, WA, USA"


In [13]:
df3.to_pickle('df_7500to10999_cleaned.pkl')

In [14]:
df = pd.read_pickle('df_7500to10999_cleaned.pkl')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2546 entries, 7500 to 10999
Data columns (total 12 columns):
OBJECTID         2546 non-null int64
WOKEY            2546 non-null int64
LOCATION         2546 non-null object
ADDRDESC         2546 non-null object
INITDT_dt        2546 non-null datetime64[ns]
FLDSTARTDT_dt    2546 non-null datetime64[ns]
FLDENDDT_dt      2546 non-null datetime64[ns]
DURATION         2546 non-null timedelta64[ns]
DURATION_td      2546 non-null float64
latitude         2546 non-null float64
longitude        2546 non-null float64
address          2546 non-null object
dtypes: datetime64[ns](3), float64(3), int64(2), object(3), timedelta64[ns](1)
memory usage: 258.6+ KB


In [16]:
df.head()

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address
7500,7501,259536,...,N 130TH ST BETWEEN MERIDIAN AVE N AND CORLISS ...,2014-02-27 08:00:00,2014-03-05 08:00:00,2014-03-05 08:00:00,6 days,6,47.710445,-122.331354,"Corliss Ave N, Seattle, WA, USA"
7502,7503,208191,1307 N Northgate Way Between Ashworth and Ston...,N NORTHGATE WAY BETWEEN ASHWORTH AVE N AND N 1...,2013-06-30 07:00:00,2013-07-02 07:00:00,2013-07-02 07:00:00,2 days,2,47.606209,-122.332071,"Seattle, WA, USA"
7503,7504,221221,NE 63rd St. between 16th and 17th NE. I ...,NE 63RD ST BETWEEN 16TH AVE NE AND 17TH AVE NE...,2013-09-03 07:00:00,2013-09-05 07:00:00,2013-09-05 07:00:00,2 days,2,47.699426,-122.309494,"17th Ave NE, Seattle, WA, USA"
7504,7505,192738,...,THOMAS ST BETWEEN MINOR AVE N AND PONTIUS AVE ...,2013-03-25 07:00:00,2013-03-28 07:00:00,2013-03-28 07:00:00,3 days,3,47.621689,-122.331715,"Pontius Ave N, Seattle, WA 98109, USA"
7505,7506,175763,4506 S Bond ...,S BOND ST BETWEEN DEAD END AND BEACON AVE S ...,2012-11-27 08:00:00,2012-11-29 08:00:00,2012-11-29 08:00:00,2 days,2,47.534119,-122.289440,"Beacon Ave S, Seattle, WA, USA"


In [17]:
df.tail()

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address
10992,10993,154426,- \n\n \n\n \n\n 49TH AVE SW BETWEEN SW EDMU...,49TH AVE SW BETWEEN SW EDMUNDS ST AND SW HUDSO...,2012-06-25 07:00:00,2012-06-25 07:00:00,2012-06-26 07:00:00,1 days,1,47.557458,-122.387559,"SW Hudson St, Seattle, WA, USA"
10993,10994,160571,"800 FAIRVIEW AVE N, ...",FAIRVIEW AVE N BETWEEN FAIRVIEW PL N AND ALOHA...,2012-08-09 07:00:00,2012-08-14 07:00:00,2012-08-14 07:00:00,5 days,5,47.627149,-122.343223,"Aloha St, Seattle, WA, USA"
10994,10995,113899,"12TH AND BOREN AVE S, ...",12TH AVE S BETWEEN BOREN AVE S AND 12TH (TURN ...,2012-04-23 07:00:00,2012-04-25 07:00:00,2012-04-25 07:00:00,2 days,2,47.606209,-122.332071,"Seattle, WA, USA"
10998,10999,96785,9719 3rd Ave NW ...,3RD AVE NW BETWEEN NW 97TH ST AND NW 100TH ST ...,2011-12-28 08:00:00,2011-12-30 08:00:00,2011-12-30 08:00:00,2 days,2,47.701542,-122.374093,"NW 100th St, Seattle, WA 98177, USA"
10999,11000,107829,Rainier Ave S to S Massachusetts ...,RAINIER AVE S BETWEEN RAINIER NB (I90 EB ON RP...,2012-03-19 07:00:00,2012-03-20 07:00:00,2012-03-20 07:00:00,1 days,1,47.588483,-122.313814,"S Massachusetts St, Seattle, WA, USA"


In [18]:
df = df[df['address'] != 'Seattle, WA, USA']

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2239 entries, 7500 to 10999
Data columns (total 12 columns):
OBJECTID         2239 non-null int64
WOKEY            2239 non-null int64
LOCATION         2239 non-null object
ADDRDESC         2239 non-null object
INITDT_dt        2239 non-null datetime64[ns]
FLDSTARTDT_dt    2239 non-null datetime64[ns]
FLDENDDT_dt      2239 non-null datetime64[ns]
DURATION         2239 non-null timedelta64[ns]
DURATION_td      2239 non-null float64
latitude         2239 non-null float64
longitude        2239 non-null float64
address          2239 non-null object
dtypes: datetime64[ns](3), float64(3), int64(2), object(3), timedelta64[ns](1)
memory usage: 227.4+ KB


In [20]:
df.head()

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address
7500,7501,259536,...,N 130TH ST BETWEEN MERIDIAN AVE N AND CORLISS ...,2014-02-27 08:00:00,2014-03-05 08:00:00,2014-03-05 08:00:00,6 days,6,47.710445,-122.331354,"Corliss Ave N, Seattle, WA, USA"
7503,7504,221221,NE 63rd St. between 16th and 17th NE. I ...,NE 63RD ST BETWEEN 16TH AVE NE AND 17TH AVE NE...,2013-09-03 07:00:00,2013-09-05 07:00:00,2013-09-05 07:00:00,2 days,2,47.699426,-122.309494,"17th Ave NE, Seattle, WA, USA"
7504,7505,192738,...,THOMAS ST BETWEEN MINOR AVE N AND PONTIUS AVE ...,2013-03-25 07:00:00,2013-03-28 07:00:00,2013-03-28 07:00:00,3 days,3,47.621689,-122.331715,"Pontius Ave N, Seattle, WA 98109, USA"
7505,7506,175763,4506 S Bond ...,S BOND ST BETWEEN DEAD END AND BEACON AVE S ...,2012-11-27 08:00:00,2012-11-29 08:00:00,2012-11-29 08:00:00,2 days,2,47.534119,-122.289440,"Beacon Ave S, Seattle, WA, USA"
7506,7507,205432,Lake WAshington Blvd from SR 520 to Lake Park ...,LAKE WASHINGTON BLVD E BETWEEN ARBORETUM DR E ...,2013-06-14 07:00:00,2013-06-18 07:00:00,2013-06-20 07:00:00,6 days,6,47.637151,-122.308871,"E Interlaken Blvd, Seattle, WA 98112, USA"


In [21]:
df.tail()

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address
10990,10991,110798,...,BOYER AVE E BETWEEN E LYNN ST AND 15TH AVE E ...,2012-03-30 07:00:00,2012-04-02 07:00:00,2012-04-02 07:00:00,3 days,3,47.630648,-122.312517,"15th Ave E, Seattle, WA, USA"
10992,10993,154426,- \n\n \n\n \n\n 49TH AVE SW BETWEEN SW EDMU...,49TH AVE SW BETWEEN SW EDMUNDS ST AND SW HUDSO...,2012-06-25 07:00:00,2012-06-25 07:00:00,2012-06-26 07:00:00,1 days,1,47.557458,-122.387559,"SW Hudson St, Seattle, WA, USA"
10993,10994,160571,"800 FAIRVIEW AVE N, ...",FAIRVIEW AVE N BETWEEN FAIRVIEW PL N AND ALOHA...,2012-08-09 07:00:00,2012-08-14 07:00:00,2012-08-14 07:00:00,5 days,5,47.627149,-122.343223,"Aloha St, Seattle, WA, USA"
10998,10999,96785,9719 3rd Ave NW ...,3RD AVE NW BETWEEN NW 97TH ST AND NW 100TH ST ...,2011-12-28 08:00:00,2011-12-30 08:00:00,2011-12-30 08:00:00,2 days,2,47.701542,-122.374093,"NW 100th St, Seattle, WA 98177, USA"
10999,11000,107829,Rainier Ave S to S Massachusetts ...,RAINIER AVE S BETWEEN RAINIER NB (I90 EB ON RP...,2012-03-19 07:00:00,2012-03-20 07:00:00,2012-03-20 07:00:00,1 days,1,47.588483,-122.313814,"S Massachusetts St, Seattle, WA, USA"


In [22]:
df.to_pickle('df_7500to10999_geo_cleaned.pkl')

In [23]:
df = pd.read_pickle('df_7500to10999_geo_cleaned.pkl')

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2239 entries, 7500 to 10999
Data columns (total 12 columns):
OBJECTID         2239 non-null int64
WOKEY            2239 non-null int64
LOCATION         2239 non-null object
ADDRDESC         2239 non-null object
INITDT_dt        2239 non-null datetime64[ns]
FLDSTARTDT_dt    2239 non-null datetime64[ns]
FLDENDDT_dt      2239 non-null datetime64[ns]
DURATION         2239 non-null timedelta64[ns]
DURATION_td      2239 non-null float64
latitude         2239 non-null float64
longitude        2239 non-null float64
address          2239 non-null object
dtypes: datetime64[ns](3), float64(3), int64(2), object(3), timedelta64[ns](1)
memory usage: 227.4+ KB


In [25]:
df.head()

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address
7500,7501,259536,...,N 130TH ST BETWEEN MERIDIAN AVE N AND CORLISS ...,2014-02-27 08:00:00,2014-03-05 08:00:00,2014-03-05 08:00:00,6 days,6,47.710445,-122.331354,"Corliss Ave N, Seattle, WA, USA"
7503,7504,221221,NE 63rd St. between 16th and 17th NE. I ...,NE 63RD ST BETWEEN 16TH AVE NE AND 17TH AVE NE...,2013-09-03 07:00:00,2013-09-05 07:00:00,2013-09-05 07:00:00,2 days,2,47.699426,-122.309494,"17th Ave NE, Seattle, WA, USA"
7504,7505,192738,...,THOMAS ST BETWEEN MINOR AVE N AND PONTIUS AVE ...,2013-03-25 07:00:00,2013-03-28 07:00:00,2013-03-28 07:00:00,3 days,3,47.621689,-122.331715,"Pontius Ave N, Seattle, WA 98109, USA"
7505,7506,175763,4506 S Bond ...,S BOND ST BETWEEN DEAD END AND BEACON AVE S ...,2012-11-27 08:00:00,2012-11-29 08:00:00,2012-11-29 08:00:00,2 days,2,47.534119,-122.289440,"Beacon Ave S, Seattle, WA, USA"
7506,7507,205432,Lake WAshington Blvd from SR 520 to Lake Park ...,LAKE WASHINGTON BLVD E BETWEEN ARBORETUM DR E ...,2013-06-14 07:00:00,2013-06-18 07:00:00,2013-06-20 07:00:00,6 days,6,47.637151,-122.308871,"E Interlaken Blvd, Seattle, WA 98112, USA"


In [26]:
df.tail()

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address
10990,10991,110798,...,BOYER AVE E BETWEEN E LYNN ST AND 15TH AVE E ...,2012-03-30 07:00:00,2012-04-02 07:00:00,2012-04-02 07:00:00,3 days,3,47.630648,-122.312517,"15th Ave E, Seattle, WA, USA"
10992,10993,154426,- \n\n \n\n \n\n 49TH AVE SW BETWEEN SW EDMU...,49TH AVE SW BETWEEN SW EDMUNDS ST AND SW HUDSO...,2012-06-25 07:00:00,2012-06-25 07:00:00,2012-06-26 07:00:00,1 days,1,47.557458,-122.387559,"SW Hudson St, Seattle, WA, USA"
10993,10994,160571,"800 FAIRVIEW AVE N, ...",FAIRVIEW AVE N BETWEEN FAIRVIEW PL N AND ALOHA...,2012-08-09 07:00:00,2012-08-14 07:00:00,2012-08-14 07:00:00,5 days,5,47.627149,-122.343223,"Aloha St, Seattle, WA, USA"
10998,10999,96785,9719 3rd Ave NW ...,3RD AVE NW BETWEEN NW 97TH ST AND NW 100TH ST ...,2011-12-28 08:00:00,2011-12-30 08:00:00,2011-12-30 08:00:00,2 days,2,47.701542,-122.374093,"NW 100th St, Seattle, WA 98177, USA"
10999,11000,107829,Rainier Ave S to S Massachusetts ...,RAINIER AVE S BETWEEN RAINIER NB (I90 EB ON RP...,2012-03-19 07:00:00,2012-03-20 07:00:00,2012-03-20 07:00:00,1 days,1,47.588483,-122.313814,"S Massachusetts St, Seattle, WA, USA"


In [27]:
df = pd.read_pickle('df_1to7499_geo_cleaned.pkl')

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4757 entries, 0 to 7499
Data columns (total 12 columns):
OBJECTID         4757 non-null int64
WOKEY            4757 non-null int64
LOCATION         4757 non-null object
ADDRDESC         4757 non-null object
INITDT_dt        4757 non-null datetime64[ns]
FLDSTARTDT_dt    4757 non-null datetime64[ns]
FLDENDDT_dt      4757 non-null datetime64[ns]
DURATION         4757 non-null timedelta64[ns]
DURATION_td      4757 non-null float64
latitude         4757 non-null float64
longitude        4757 non-null float64
address          4757 non-null object
dtypes: datetime64[ns](3), float64(3), int64(2), object(3), timedelta64[ns](1)
memory usage: 483.1+ KB


In [29]:
df = pd.read_pickle('df_1to7499_features.pkl')

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4757 entries, 0 to 4756
Data columns (total 32 columns):
OBJECTID                  4757 non-null int64
WOKEY                     4757 non-null int64
LOCATION                  4757 non-null object
ADDRDESC                  4757 non-null object
INITDT_dt                 4757 non-null datetime64[ns]
FLDSTARTDT_dt             4757 non-null datetime64[ns]
FLDENDDT_dt               4757 non-null datetime64[ns]
DURATION                  4757 non-null timedelta64[ns]
DURATION_td               4757 non-null float64
latitude                  4757 non-null float64
longitude                 4757 non-null float64
address                   4757 non-null object
Seattle_dist              4757 non-null float64
Space_Needle_dist         4757 non-null float64
Pike_Place_dist           4757 non-null float64
Convention_Center_dist    4757 non-null float64
Woodland_Park_dist        4757 non-null float64
Queene_Anne_dist          4757 non-null float64
INIT_Qu

In [31]:
df.head(2)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,...,GEOID,GEO.id,GEO.display-label,Median_Value,Margin_of_Error,SND_FEACOD,ST_CODE,SEGMENT_TY,DIVIDED_CO,VEHICLE_US
0,1,2456,Holden pl SW bt SW Kenyon Holden ...,HOLDEN PL SW BETWEEN SW HOLDEN ST AND SW KENYO...,2010-03-29 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,8 days,8,47.531858,...,530330114011,1500000US530330114011,"Block Group 1, Census Tract 114.01, King Count...",307500,53269,1,0,1,1,1
1,2,2514,"NE Pacific Street, one block west of Montlake ...",MONTLAKE BLVD NE AND NE PACIFIC ST ...,2010-03-30 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,2 days,2,47.649014,...,530330053022,1500000US530330053022,"Block Group 2, Census Tract 53.02, King County...",NaN,NaN,5,0,1,1,1


In [32]:
df.tail(2)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,...,GEOID,GEO.id,GEO.display-label,Median_Value,Margin_of_Error,SND_FEACOD,ST_CODE,SEGMENT_TY,DIVIDED_CO,VEHICLE_US
4755,7499,225583,NE 130TH ST BETWEEN LAKE CITY WAY NE AND 33RD ...,NE 130TH ST BETWEEN LAKE CITY WAY NE AND 33RD ...,2013-10-01 07:00:00,2013-10-01 07:00:00,2013-10-02 07:00:00,1 days,1,47.744614,...,530330214002,1500000US530330214002,"Block Group 2, Census Tract 214, King County, ...",428400,54553,1,0,1,1,1
4756,7500,210335,EB bike lane between 14th and 16th Ave ...,E UNION ST BETWEEN 15TH AVE AND 16TH AVE ...,2013-07-10 07:00:00,2013-07-12 07:00:00,2013-07-12 07:00:00,2 days,2,47.610378,...,530330079003,1500000US530330079003,"Block Group 3, Census Tract 79, King County, W...",245000,35437,NaN,NaN,NaN,NaN,NaN


####The featurizing process re-indexes the rows to start again from zero.

####Look at the rows, 7500-10999 featurized overnight

In [33]:
pd.set_option('display.max_columns', 500)

In [34]:
df = pd.read_pickle('df_1to7499_features.pkl')

In [35]:
df2 = pd.read_pickle('df_7500to10999_features.pkl')

In [36]:
df.tail(2)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,INIT_month,neighborhood_label,GEOID,GEO.id,GEO.display-label,Median_Value,Margin_of_Error,SND_FEACOD,ST_CODE,SEGMENT_TY,DIVIDED_CO,VEHICLE_US
4755,7499,225583,NE 130TH ST BETWEEN LAKE CITY WAY NE AND 33RD ...,NE 130TH ST BETWEEN LAKE CITY WAY NE AND 33RD ...,2013-10-01 07:00:00,2013-10-01 07:00:00,2013-10-02 07:00:00,1 days,1,47.744614,-122.294066,"33rd Ave NE, Seattle, WA, USA",9.724967,8.952767,9.586114,9.369915,5.944939,7.978499,4,-4,10,,530330214002,1500000US530330214002,"Block Group 2, Census Tract 214, King County, ...",428400,54553,1,0,1,1,1
4756,7500,210335,EB bike lane between 14th and 16th Ave ...,E UNION ST BETWEEN 15TH AVE AND 16TH AVE ...,2013-07-10 07:00:00,2013-07-12 07:00:00,2013-07-12 07:00:00,2 days,2,47.610378,-122.311513,"16th Ave, Seattle, WA 98122, USA",1.002700,1.897800,1.434032,0.944730,4.473846,2.862636,3,-1,7,55,530330079003,1500000US530330079003,"Block Group 3, Census Tract 79, King County, W...",245000,35437,NaN,NaN,NaN,NaN,NaN


In [37]:
df2.head(2)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,INIT_month,neighborhood_label,GEOID,GEO.id,GEO.display-label,Median_Value,Margin_of_Error,SND_FEACOD,ST_CODE,SEGMENT_TY,DIVIDED_CO,VEHICLE_US
0,7501,259536,...,N 130TH ST BETWEEN MERIDIAN AVE N AND CORLISS ...,2014-02-27 08:00:00,2014-03-05 08:00:00,2014-03-05 08:00:00,6 days,6,47.710445,-122.331354,"Corliss Ave N, Seattle, WA, USA",7.201347,6.269615,6.977110,6.843456,3.076210,5.191419,1,4,2,112,530330012005,1500000US530330012005,"Block Group 5, Census Tract 12, King County, W...",200800,41955,NaN,NaN,NaN,NaN,NaN
1,7504,221221,NE 63rd St. between 16th and 17th NE. I ...,NE 63RD ST BETWEEN 16TH AVE NE AND 17TH AVE NE...,2013-09-03 07:00:00,2013-09-05 07:00:00,2013-09-05 07:00:00,2 days,2,47.699426,-122.309494,"17th Ave NE, Seattle, WA, USA",6.525654,5.759803,6.382659,6.169696,2.965802,4.839590,3,-3,9,116,530330020001,1500000US530330020001,"Block Group 1, Census Tract 20, King County, W...",382700,29634,NaN,NaN,NaN,NaN,NaN


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4757 entries, 0 to 4756
Data columns (total 32 columns):
OBJECTID                  4757 non-null int64
WOKEY                     4757 non-null int64
LOCATION                  4757 non-null object
ADDRDESC                  4757 non-null object
INITDT_dt                 4757 non-null datetime64[ns]
FLDSTARTDT_dt             4757 non-null datetime64[ns]
FLDENDDT_dt               4757 non-null datetime64[ns]
DURATION                  4757 non-null timedelta64[ns]
DURATION_td               4757 non-null float64
latitude                  4757 non-null float64
longitude                 4757 non-null float64
address                   4757 non-null object
Seattle_dist              4757 non-null float64
Space_Needle_dist         4757 non-null float64
Pike_Place_dist           4757 non-null float64
Convention_Center_dist    4757 non-null float64
Woodland_Park_dist        4757 non-null float64
Queene_Anne_dist          4757 non-null float64
INIT_Qu

In [39]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2239 entries, 0 to 2238
Data columns (total 32 columns):
OBJECTID                  2239 non-null int64
WOKEY                     2239 non-null int64
LOCATION                  2239 non-null object
ADDRDESC                  2239 non-null object
INITDT_dt                 2239 non-null datetime64[ns]
FLDSTARTDT_dt             2239 non-null datetime64[ns]
FLDENDDT_dt               2239 non-null datetime64[ns]
DURATION                  2239 non-null timedelta64[ns]
DURATION_td               2239 non-null float64
latitude                  2239 non-null float64
longitude                 2239 non-null float64
address                   2239 non-null object
Seattle_dist              2239 non-null float64
Space_Needle_dist         2239 non-null float64
Pike_Place_dist           2239 non-null float64
Convention_Center_dist    2239 non-null float64
Woodland_Park_dist        2239 non-null float64
Queene_Anne_dist          2239 non-null float64
INIT_Qu

In [40]:
df.columns

Index([u'OBJECTID', u'WOKEY', u'LOCATION', u'ADDRDESC', u'INITDT_dt',
       u'FLDSTARTDT_dt', u'FLDENDDT_dt', u'DURATION', u'DURATION_td',
       u'latitude', u'longitude', u'address', u'Seattle_dist',
       u'Space_Needle_dist', u'Pike_Place_dist', u'Convention_Center_dist',
       u'Woodland_Park_dist', u'Queene_Anne_dist', u'INIT_Quarter',
       u'days_end_FY', u'INIT_month', u'neighborhood_label', u'GEOID',
       u'GEO.id', u'GEO.display-label', u'Median_Value', u'Margin_of_Error',
       u'SND_FEACOD', u'ST_CODE', u'SEGMENT_TY', u'DIVIDED_CO', u'VEHICLE_US'],
      dtype='object')

In [41]:
df2.columns

Index([u'OBJECTID', u'WOKEY', u'LOCATION', u'ADDRDESC', u'INITDT_dt',
       u'FLDSTARTDT_dt', u'FLDENDDT_dt', u'DURATION', u'DURATION_td',
       u'latitude', u'longitude', u'address', u'Seattle_dist',
       u'Space_Needle_dist', u'Pike_Place_dist', u'Convention_Center_dist',
       u'Woodland_Park_dist', u'Queene_Anne_dist', u'INIT_Quarter',
       u'days_end_FY', u'INIT_month', u'neighborhood_label', u'GEOID',
       u'GEO.id', u'GEO.display-label', u'Median_Value', u'Margin_of_Error',
       u'SND_FEACOD', u'ST_CODE', u'SEGMENT_TY', u'DIVIDED_CO', u'VEHICLE_US'],
      dtype='object')

In [42]:
df.columns.tolist() == df2.columns.tolist()

True

In [43]:
df.tail()

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,INIT_month,neighborhood_label,GEOID,GEO.id,GEO.display-label,Median_Value,Margin_of_Error,SND_FEACOD,ST_CODE,SEGMENT_TY,DIVIDED_CO,VEHICLE_US
4752,7490,222968,"Gar cemetery, Everett and Howe street, capitol...",E NEWTON ST BETWEEN 12TH AVE E AND EVERETT AVE...,2013-09-11 07:00:00,2013-09-12 07:00:00,2013-09-12 07:00:00,1 days,1,47.639113,-122.315199,"Everett Ave E, Seattle, WA, USA",2.405867,2.045775,2.390182,2.064206,2.711767,1.996535,3,-3,9,98,530330062003,1500000US530330062003,"Block Group 3, Census Tract 62, King County, W...",907700,107139,NaN,NaN,NaN,NaN,NaN
4753,7495,227737,...,NE 43RD ST BETWEEN ROOSEVELT (N LEG) WAY NE AN...,2013-10-11 07:00:00,2013-10-18 07:00:00,2013-10-18 07:00:00,7 days,7,47.717633,-122.316107,"11th Ave NE, Seattle, WA, USA",7.733851,6.886396,7.554102,7.375943,3.817021,5.871300,4,-4,10,113,530330006006,1500000US530330006006,"Block Group 6, Census Tract 6, King County, Wa...",342400,30558,1,0,1,1,1
4754,7498,257523,3rd Ave NW by intersection of NW 56th ...,3RD AVE NW AND NW 56TH ST ...,2014-02-21 08:00:00,2014-02-27 08:00:00,2014-02-27 08:00:00,6 days,6,47.669437,-122.360798,"3rd Ave NW & NW 56th St, Seattle, WA 98107, USA",4.569466,3.422965,4.216090,4.234508,0.339532,2.212660,1,4,2,31,530330034003,1500000US530330034003,"Block Group 3, Census Tract 34, King County, W...",491700,56071,77,22,8,1,2
4755,7499,225583,NE 130TH ST BETWEEN LAKE CITY WAY NE AND 33RD ...,NE 130TH ST BETWEEN LAKE CITY WAY NE AND 33RD ...,2013-10-01 07:00:00,2013-10-01 07:00:00,2013-10-02 07:00:00,1 days,1,47.744614,-122.294066,"33rd Ave NE, Seattle, WA, USA",9.724967,8.952767,9.586114,9.369915,5.944939,7.978499,4,-4,10,,530330214002,1500000US530330214002,"Block Group 2, Census Tract 214, King County, ...",428400,54553,1,0,1,1,1
4756,7500,210335,EB bike lane between 14th and 16th Ave ...,E UNION ST BETWEEN 15TH AVE AND 16TH AVE ...,2013-07-10 07:00:00,2013-07-12 07:00:00,2013-07-12 07:00:00,2 days,2,47.610378,-122.311513,"16th Ave, Seattle, WA 98122, USA",1.002700,1.897800,1.434032,0.944730,4.473846,2.862636,3,-1,7,55,530330079003,1500000US530330079003,"Block Group 3, Census Tract 79, King County, W...",245000,35437,NaN,NaN,NaN,NaN,NaN


In [44]:
df2.head()

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,INIT_month,neighborhood_label,GEOID,GEO.id,GEO.display-label,Median_Value,Margin_of_Error,SND_FEACOD,ST_CODE,SEGMENT_TY,DIVIDED_CO,VEHICLE_US
0,7501,259536,...,N 130TH ST BETWEEN MERIDIAN AVE N AND CORLISS ...,2014-02-27 08:00:00,2014-03-05 08:00:00,2014-03-05 08:00:00,6 days,6,47.710445,-122.331354,"Corliss Ave N, Seattle, WA, USA",7.201347,6.269615,6.977110,6.843456,3.076210,5.191419,1,4,2,112,530330012005,1500000US530330012005,"Block Group 5, Census Tract 12, King County, W...",200800,41955,NaN,NaN,NaN,NaN,NaN
1,7504,221221,NE 63rd St. between 16th and 17th NE. I ...,NE 63RD ST BETWEEN 16TH AVE NE AND 17TH AVE NE...,2013-09-03 07:00:00,2013-09-05 07:00:00,2013-09-05 07:00:00,2 days,2,47.699426,-122.309494,"17th Ave NE, Seattle, WA, USA",6.525654,5.759803,6.382659,6.169696,2.965802,4.839590,3,-3,9,116,530330020001,1500000US530330020001,"Block Group 1, Census Tract 20, King County, W...",382700,29634,NaN,NaN,NaN,NaN,NaN
2,7505,192738,...,THOMAS ST BETWEEN MINOR AVE N AND PONTIUS AVE ...,2013-03-25 07:00:00,2013-03-28 07:00:00,2013-03-28 07:00:00,3 days,3,47.621689,-122.331715,"Pontius Ave N, Seattle, WA 98109, USA",1.069515,0.824362,0.960866,0.711558,3.394808,1.637910,1,3,3,49,530330073001,1500000US530330073001,"Block Group 1, Census Tract 73, King County, W...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7506,175763,4506 S Bond ...,S BOND ST BETWEEN DEAD END AND BEACON AVE S ...,2012-11-27 08:00:00,2012-11-29 08:00:00,2012-11-29 08:00:00,2 days,2,47.534119,-122.289440,"Beacon Ave S, Seattle, WA, USA",5.364434,6.591131,5.775969,5.691800,9.759099,7.824063,4,-5,11,81,530330110013,1500000US530330110013,"Block Group 3, Census Tract 110.01, King Count...",303900,50586,NaN,NaN,NaN,NaN,NaN
4,7507,205432,Lake WAshington Blvd from SR 520 to Lake Park ...,LAKE WASHINGTON BLVD E BETWEEN ARBORETUM DR E ...,2013-06-14 07:00:00,2013-06-18 07:00:00,2013-06-20 07:00:00,6 days,6,47.637151,-122.308871,"E Interlaken Blvd, Seattle, WA 98112, USA",2.396575,2.209793,2.452252,2.074241,3.011768,2.288921,2,0,6,98,530330062003,1500000US530330062003,"Block Group 3, Census Tract 62, King County, W...",907700,107139,NaN,NaN,NaN,NaN,NaN


####OK, append

In [45]:
df3 = df.append(df2)

In [46]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6996 entries, 0 to 2238
Data columns (total 32 columns):
OBJECTID                  6996 non-null int64
WOKEY                     6996 non-null int64
LOCATION                  6996 non-null object
ADDRDESC                  6996 non-null object
INITDT_dt                 6996 non-null datetime64[ns]
FLDSTARTDT_dt             6996 non-null datetime64[ns]
FLDENDDT_dt               6996 non-null datetime64[ns]
DURATION                  6996 non-null timedelta64[ns]
DURATION_td               6996 non-null float64
latitude                  6996 non-null float64
longitude                 6996 non-null float64
address                   6996 non-null object
Seattle_dist              6996 non-null float64
Space_Needle_dist         6996 non-null float64
Pike_Place_dist           6996 non-null float64
Convention_Center_dist    6996 non-null float64
Woodland_Park_dist        6996 non-null float64
Queene_Anne_dist          6996 non-null float64
INIT_Qu

In [47]:
df3.tail(25)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,INIT_month,neighborhood_label,GEOID,GEO.id,GEO.display-label,Median_Value,Margin_of_Error,SND_FEACOD,ST_CODE,SEGMENT_TY,DIVIDED_CO,VEHICLE_US
2214,10957,199216,pothole in N lane of Meridian Ave forces bicyc...,MERIDIAN AVE N BETWEEN N 50TH ST AND N 51ST ST...,2013-05-13 07:00:00,2013-05-14 07:00:00,2013-05-14 07:00:00,1 days 00:00:00,1,47.665735,-122.340108,"N 51st St, Seattle, WA 98103, USA",4.129457,3.153856,3.871068,3.775105,0.660785,2.121796,2,1,5,31,530330046001,1500000US530330046001,"Block Group 1, Census Tract 46, King County, W...",704800,97261,NaN,NaN,NaN,NaN,NaN
2215,10960,118729,...,34TH AVE AND E PIKE ST ...,2012-05-21 07:00:00,2012-05-22 07:00:00,2012-05-22 07:00:00,1 days 00:00:00,1,47.614052,-122.289252,"34th Ave & E Pike St, Seattle, WA 98122, USA",2.072516,2.839149,2.491291,1.990603,4.817602,3.590751,2,1,5,53,530330078003,1500000US530330078003,"Block Group 3, Census Tract 78, King County, W...",650700,101638,NaN,NaN,NaN,NaN,NaN
2216,10961,119541,...,SW ROXBURY ST BETWEEN 30TH AVE SW AND 31ST PL ...,2012-05-29 07:00:00,2012-05-30 07:00:00,2012-05-30 07:00:00,1 days 00:00:00,1,47.519743,-122.373381,"31st Pl SW, Seattle, WA 98126, USA",6.278128,7.051905,6.384866,6.624842,10.320995,8.165412,2,1,5,94,530330114023,1500000US530330114023,"Block Group 3, Census Tract 114.02, King Count...",273200,24636,NaN,NaN,NaN,NaN,NaN
2217,10963,159493,1211 MINOR AVE ...,MINOR AVE BETWEEN SENECA ST AND UNIVERSITY ST ...,2012-07-31 07:00:00,2012-08-03 07:00:00,2012-08-03 07:00:00,3 days 00:00:00,3,47.609386,-122.332198,"University St, Seattle, WA 98101, USA",0.219558,1.107590,0.467537,0.140449,4.207558,2.281253,3,-1,7,62,530330082001,1500000US530330082001,"Block Group 1, Census Tract 82, King County, W...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2218,10964,163278,SW Roxbury St and 47th Ave SW intersection the...,S ROXBURY ST BETWEEN MARCUS AVE S AND 48TH AVE...,2012-09-02 07:00:00,2012-09-05 07:00:00,2012-09-05 07:00:00,3 days 00:00:00,3,47.505143,-122.273783,"48th Ave S, Seattle, WA, USA",7.495465,8.716740,7.901378,7.823483,11.889464,9.951672,3,-3,9,,530330263001,1500000US530330263001,"Block Group 1, Census Tract 263, King County, ...",260000,40855,NaN,NaN,NaN,NaN,NaN
2219,10965,162384,...,W RAYE ST BETWEEN DEAD END 3 AND 23RD AVE W ...,2012-08-24 07:00:00,2012-08-29 07:00:00,2012-08-29 07:00:00,5 days 00:00:00,5,47.645996,-122.385913,"23rd Ave W, Seattle, WA, USA",3.725391,2.455147,3.232606,3.482954,2.166610,1.434918,3,-2,8,52,530330058021,1500000US530330058021,"Block Group 1, Census Tract 58.02, King County...",476300,99226,NaN,NaN,NaN,NaN,NaN
2220,10972,160024,"8th Ave. NW., betweeen NW 65th and Market sts ...",8TH AVE NW AND NW 65TH ST ...,2012-08-05 07:00:00,2012-08-07 07:00:00,2012-08-07 07:00:00,2 days 00:00:00,2,47.675980,-122.366063,"8th Ave NW & NW 65th St, Seattle, WA 98117, USA",5.074751,3.911766,4.711365,4.742314,0.774636,2.687739,3,-2,8,30,530330033002,1500000US530330033002,"Block Group 2, Census Tract 33, King County, W...",395800,78957,NaN,NaN,NaN,NaN,NaN
2221,10974,165468,2415 NE 80th St crosswalk ...,NE 80TH ST BETWEEN 24TH AVE NE AND 25TH AVE NE...,2012-09-18 07:00:00,2012-09-19 07:00:00,2012-09-19 07:00:00,1 days 00:00:00,1,47.717509,-122.301791,"25th Ave NE, Seattle, WA, USA",7.818066,7.058444,7.681765,7.462968,4.158806,6.117796,3,-3,9,102,530330007002,1500000US530330007002,"Block Group 2, Census Tract 7, King County, Wa...",293500,48319,NaN,NaN,NaN,NaN,NaN
2222,10976,111842,ROOSEVELT WY NE BTW 65TH AVE NE & 75TH AVE NE ...,ROOSEVELT WAY NE BETWEEN NE 65TH ST AND NE 66T...,2012-04-06 07:00:00,2012-04-11 07:00:00,2012-04-23 07:00:00,17 days 00:00:00,17,47.676580,-122.315665,"NE 66th St, Seattle, WA 98115, USA",4.921630,4.179689,4.782303,4.565471,1.857281,3.344408,2,2,4,41,5303300

In [48]:
df3.head(25)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,INIT_month,neighborhood_label,GEOID,GEO.id,GEO.display-label,Median_Value,Margin_of_Error,SND_FEACOD,ST_CODE,SEGMENT_TY,DIVIDED_CO,VEHICLE_US
0,1,2456,Holden pl SW bt SW Kenyon Holden ...,HOLDEN PL SW BETWEEN SW HOLDEN ST AND SW KENYO...,2010-03-29 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,8 days,8,47.531858,-122.363186,"SW Kenyon St, Seattle, WA, USA",5.338641,6.158729,5.467939,5.688451,9.453253,7.300465,1,3,3,93,530330114011,1500000US530330114011,"Block Group 1, Census Tract 114.01, King Count...",307500,53269,1,0,1,1,1
1,2,2514,"NE Pacific Street, one block west of Montlake ...",MONTLAKE BLVD NE AND NE PACIFIC ST ...,2010-03-30 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,2 days,2,47.649014,-122.304655,"Montlake Blvd NE & NE Pacific St, Seattle, WA ...",3.222458,2.867154,3.231487,2.889531,2.654837,2.610214,1,3,3,42,530330053022,1500000US530330053022,"Block Group 2, Census Tract 53.02, King County...",NaN,NaN,5,0,1,1,1
2,3,2649,32nd Ave NE and NE 135th ...,NE 135TH ST BETWEEN 32ND AVE NE AND LAKE CITY ...,2010-03-31 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,1 days,1,47.707446,-122.301566,"Lake City Way NE, Seattle, WA, USA",7.137526,6.405817,7.012778,6.783446,3.623119,5.502601,1,3,3,104,530330011001,1500000US530330011001,"Block Group 1, Census Tract 11, King County, W...",416900,62441,5,0,1,1,1
3,4,1591,Charles st & 6 Ave S pothole next to traffic l...,S CHARLES ST BETWEEN 6TH AVE S AND MAYNARD AVE...,2010-03-15 07:00:00,2010-03-16 07:00:00,2010-03-16 07:00:00,1 days,1,47.597085,-122.325065,"Maynard Ave S, Seattle, WA, USA",0.710334,1.974308,1.184267,1.035443,5.113765,3.183602,1,3,3,61,530330091002,1500000US530330091002,"Block Group 2, Census Tract 91, King County, W...",328900,145688,1,0,1,1,1
4,6,5642,52nd St. & 18th Ave. NE ...,18TH AVE NE BETWEEN NE 52ND ST AND NE 55TH ST ...,2010-04-19 07:00:00,2010-04-22 07:00:00,2010-04-22 07:00:00,3 days,3,47.668522,-122.295630,"NE 55th St, Seattle, WA 98105, USA",4.628992,4.156594,4.607589,4.291005,2.707355,3.612252,2,2,4,36,530330042005,1500000US530330042005,"Block Group 5, Census Tract 42, King County, W...",637000,56463,NaN,NaN,NaN,NaN,NaN
5,7,7585,...,3RD AVE NW BETWEEN NW 125TH ST AND NW 127TH ST...,2010-05-03 07:00:00,2010-05-04 07:00:00,2010-05-04 07:00:00,1 days,1,47.721476,-122.364984,"NW 127th St, Seattle, WA 98177, USA",8.110129,7.014027,7.793727,7.762688,3.695250,5.813108,2,1,5,110,530330005002,1500000US530330005002,"Block Group 2, Census Tract 5, King County, Wa...",565200,53534,5,0,1,1,1
6,10,7211,Republican and Minor Ave N ...,MINOR AVE N AND REPUBLICAN ST ...,2010-04-29 07:00:00,2010-05-03 07:00:00,2010-05-03 07:00:00,4 days,4,47.623131,-122.333003,"Republican St & Minor Ave N, Seattle, WA 98109...",1.169861,0.781462,1.021139,0.813571,3.281758,1.527051,2,2,4,49,530330073001,1500000US530330073001,"Block Group 1, Census Tract 73, King County, W...",NaN,NaN,1,0,1,1,1
7,11,3168,...,17TH AVE W BETWEEN W DRAVUS ST AND W BERTONA S...,2010-04-02 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,4 days,4,47.650311,-122.388604,"W Bertona St, Seattle, WA, USA",4.031574,2.758997,3.544222,3.781162,2.061169,1.686262,2,2,4,50,530330058012,1500000US530330058012,"Block Group 2, Census Tract 58.01, King County...",255200,50905,NaN,NaN,NaN,NaN,NaN
8,13,6967,B/T Pike and Pine B/t 211 and 215 ...,PIKE ST BETWEEN 2ND AVE AND 3RD AVE ...,2010-04-28 07:00:00,2010-05-03 07:00:00,2010-05-03 07:00:00,5 days,5,47.610741,-122.338703,"3rd Ave, Seattle, WA, USA",0.440475,0.836150,0.177650,0.331137,4.053555,2.052527,2,2,4,62,530330081002,1500000US530330081002,"Block Group 2, Census Tract 81, King County, W...",384400,224247,NaN,NaN,NaN,NaN,NaN
9,14,7107,17th Ave NE Exit of University of Washington ...,NE 45TH ST BETWEEN 15TH AVE NE AND 1

In [49]:
len(df3.index.tolist())

6996

In [50]:
len(set(df3.index.tolist()))

4757

####There are duplicate indices; reset index

In [51]:
df3.reset_index(drop=True)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,INIT_month,neighborhood_label,GEOID,GEO.id,GEO.display-label,Median_Value,Margin_of_Error,SND_FEACOD,ST_CODE,SEGMENT_TY,DIVIDED_CO,VEHICLE_US
0,1,2456,Holden pl SW bt SW Kenyon Holden ...,HOLDEN PL SW BETWEEN SW HOLDEN ST AND SW KENYO...,2010-03-29 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,8 days 00:00:00,8,47.531858,-122.363186,"SW Kenyon St, Seattle, WA, USA",5.338641,6.158729,5.467939,5.688451,9.453253,7.300465,1,3,3,93,530330114011,1500000US530330114011,"Block Group 1, Census Tract 114.01, King Count...",307500,53269,1,0,1,1,1
1,2,2514,"NE Pacific Street, one block west of Montlake ...",MONTLAKE BLVD NE AND NE PACIFIC ST ...,2010-03-30 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,2 days 00:00:00,2,47.649014,-122.304655,"Montlake Blvd NE & NE Pacific St, Seattle, WA ...",3.222458,2.867154,3.231487,2.889531,2.654837,2.610214,1,3,3,42,530330053022,1500000US530330053022,"Block Group 2, Census Tract 53.02, King County...",NaN,NaN,5,0,1,1,1
2,3,2649,32nd Ave NE and NE 135th ...,NE 135TH ST BETWEEN 32ND AVE NE AND LAKE CITY ...,2010-03-31 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,1 days 00:00:00,1,47.707446,-122.301566,"Lake City Way NE, Seattle, WA, USA",7.137526,6.405817,7.012778,6.783446,3.623119,5.502601,1,3,3,104,530330011001,1500000US530330011001,"Block Group 1, Census Tract 11, King County, W...",416900,62441,5,0,1,1,1
3,4,1591,Charles st & 6 Ave S pothole next to traffic l...,S CHARLES ST BETWEEN 6TH AVE S AND MAYNARD AVE...,2010-03-15 07:00:00,2010-03-16 07:00:00,2010-03-16 07:00:00,1 days 00:00:00,1,47.597085,-122.325065,"Maynard Ave S, Seattle, WA, USA",0.710334,1.974308,1.184267,1.035443,5.113765,3.183602,1,3,3,61,530330091002,1500000US530330091002,"Block Group 2, Census Tract 91, King County, W...",328900,145688,1,0,1,1,1
4,6,5642,52nd St. & 18th Ave. NE ...,18TH AVE NE BETWEEN NE 52ND ST AND NE 55TH ST ...,2010-04-19 07:00:00,2010-04-22 07:00:00,2010-04-22 07:00:00,3 days 00:00:00,3,47.668522,-122.295630,"NE 55th St, Seattle, WA 98105, USA",4.628992,4.156594,4.607589,4.291005,2.707355,3.612252,2,2,4,36,530330042005,1500000US530330042005,"Block Group 5, Census Tract 42, King County, W...",637000,56463,NaN,NaN,NaN,NaN,NaN
5,7,7585,...,3RD AVE NW BETWEEN NW 125TH ST AND NW 127TH ST...,2010-05-03 07:00:00,2010-05-04 07:00:00,2010-05-04 07:00:00,1 days 00:00:00,1,47.721476,-122.364984,"NW 127th St, Seattle, WA 98177, USA",8.110129,7.014027,7.793727,7.762688,3.695250,5.813108,2,1,5,110,530330005002,1500000US530330005002,"Block Group 2, Census Tract 5, King County, Wa...",565200,53534,5,0,1,1,1
6,10,7211,Republican and Minor Ave N ...,MINOR AVE N AND REPUBLICAN ST ...,2010-04-29 07:00:00,2010-05-03 07:00:00,2010-05-03 07:00:00,4 days 00:00:00,4,47.623131,-122.333003,"Republican St & Minor Ave N, Seattle, WA 98109...",1.169861,0.781462,1.021139,0.813571,3.281758,1.527051,2,2,4,49,530330073001,1500000US530330073001,"Block Group 1, Census Tract 73, King County, W...",NaN,NaN,1,0,1,1,1
7,11,3168,...,17TH AVE W BETWEEN W DRAVUS ST AND W BERTONA S...,2010-04-02 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,4 days 00:00:00,4,47.650311,-122.388604,"W Bertona St, Seattle, WA, USA",4.031574,2.758997,3.544222,3.781162,2.061169,1.686262,2,2,4,50,530330058012,1500000US530330058012,"Block Group 2, Census Tract 58.01, King County...",255200,50905,NaN,NaN,NaN,NaN,NaN
8,13,6967,B/T Pike and Pine B/t 211 and 215 ...,PIKE ST BETWEEN 2ND AVE AND 3RD AVE ...,2010-04-28 07:00:00,2010-05-03 07:00:00,2010-05-03 07:00:00,5 days 00:00:00,5,47.610741,-122.338703,"3rd Ave, Seattle, WA, USA",0.440475,0.836150,0.177650,0.331137,4.053555,2.052527,2,2,4,62,530330081002,1500000US530330081002,"Block Group 2, Census Tract 81, King County, W...",384400,224247,NaN,NaN,NaN,NaN,NaN
9,14,7107,17th

####Now pickle this and go with it.  Why did the featurizing process re-index?

In [52]:
df4 = df3.reset_index(drop=True)

In [53]:
df4.to_pickle('df_1to10999_features.pkl')

In [54]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6996 entries, 0 to 6995
Data columns (total 32 columns):
OBJECTID                  6996 non-null int64
WOKEY                     6996 non-null int64
LOCATION                  6996 non-null object
ADDRDESC                  6996 non-null object
INITDT_dt                 6996 non-null datetime64[ns]
FLDSTARTDT_dt             6996 non-null datetime64[ns]
FLDENDDT_dt               6996 non-null datetime64[ns]
DURATION                  6996 non-null timedelta64[ns]
DURATION_td               6996 non-null float64
latitude                  6996 non-null float64
longitude                 6996 non-null float64
address                   6996 non-null object
Seattle_dist              6996 non-null float64
Space_Needle_dist         6996 non-null float64
Pike_Place_dist           6996 non-null float64
Convention_Center_dist    6996 non-null float64
Woodland_Park_dist        6996 non-null float64
Queene_Anne_dist          6996 non-null float64
INIT_Qu

####Examine what's happening with the index

In [158]:
df = pd.read_pickle('df_1to10999_geo_cleaned.pkl')

In [160]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6996 entries, 0 to 10999
Data columns (total 12 columns):
OBJECTID         6996 non-null int64
WOKEY            6996 non-null int64
LOCATION         6996 non-null object
ADDRDESC         6996 non-null object
INITDT_dt        6996 non-null datetime64[ns]
FLDSTARTDT_dt    6996 non-null datetime64[ns]
FLDENDDT_dt      6996 non-null datetime64[ns]
DURATION         6996 non-null timedelta64[ns]
DURATION_td      6996 non-null float64
latitude         6996 non-null float64
longitude        6996 non-null float64
address          6996 non-null object
dtypes: datetime64[ns](3), float64(3), int64(2), object(3), timedelta64[ns](1)
memory usage: 710.5+ KB


In [161]:
def _get_distance(df, origin):
    '''
    INPUT: df, tuple
    OUTPUT: pandas Series
    Input dataframe has lat-lon coords for each row/pothole.
    Returns Series containing distance from origin to each pothole.
    '''
    dists = []
    for row in df.index.tolist():
        point = df.ix[row,'latitude'], df.ix[row, 'longitude']
        dist = vincenty(origin, point)
        dists.append(dist.miles)
    dists = pd.Series(dists, index=df.index)
    return dists

In [162]:
SEATTLE_LOC = (47.6062095, -122.3320708)

In [163]:
df['Seattle_dist'] = _get_distance(df, SEATTLE_LOC)

In [164]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6996 entries, 0 to 10999
Data columns (total 13 columns):
OBJECTID         6996 non-null int64
WOKEY            6996 non-null int64
LOCATION         6996 non-null object
ADDRDESC         6996 non-null object
INITDT_dt        6996 non-null datetime64[ns]
FLDSTARTDT_dt    6996 non-null datetime64[ns]
FLDENDDT_dt      6996 non-null datetime64[ns]
DURATION         6996 non-null timedelta64[ns]
DURATION_td      6996 non-null float64
latitude         6996 non-null float64
longitude        6996 non-null float64
address          6996 non-null object
Seattle_dist     6996 non-null float64
dtypes: datetime64[ns](3), float64(4), int64(2), object(3), timedelta64[ns](1)
memory usage: 765.2+ KB


In [165]:
df.head(2)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist
0,1,2456,Holden pl SW bt SW Kenyon Holden ...,HOLDEN PL SW BETWEEN SW HOLDEN ST AND SW KENYO...,2010-03-29 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,8 days,8,47.531858,-122.363186,"SW Kenyon St, Seattle, WA, USA",5.338641
1,2,2514,"NE Pacific Street, one block west of Montlake ...",MONTLAKE BLVD NE AND NE PACIFIC ST ...,2010-03-30 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,2 days,2,47.649014,-122.304655,"Montlake Blvd NE & NE Pacific St, Seattle, WA ...",3.222458


In [166]:
df.tail(2)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist
10998,10999,96785,9719 3rd Ave NW ...,3RD AVE NW BETWEEN NW 97TH ST AND NW 100TH ST ...,2011-12-28 08:00:00,2011-12-30 08:00:00,2011-12-30 08:00:00,2 days,2,47.701542,-122.374093,"NW 100th St, Seattle, WA 98177, USA",6.872044
10999,11000,107829,Rainier Ave S to S Massachusetts ...,RAINIER AVE S BETWEEN RAINIER NB (I90 EB ON RP...,2012-03-19 07:00:00,2012-03-20 07:00:00,2012-03-20 07:00:00,1 days,1,47.588483,-122.313814,"S Massachusetts St, Seattle, WA, USA",1.492505


In [167]:
days = [6 - df.ix[row, 'INITDT_dt'].month for row in df.index.tolist()]
df['days_end_FY'] = pd.Series(days, index = df.index)

In [168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6996 entries, 0 to 10999
Data columns (total 14 columns):
OBJECTID         6996 non-null int64
WOKEY            6996 non-null int64
LOCATION         6996 non-null object
ADDRDESC         6996 non-null object
INITDT_dt        6996 non-null datetime64[ns]
FLDSTARTDT_dt    6996 non-null datetime64[ns]
FLDENDDT_dt      6996 non-null datetime64[ns]
DURATION         6996 non-null timedelta64[ns]
DURATION_td      6996 non-null float64
latitude         6996 non-null float64
longitude        6996 non-null float64
address          6996 non-null object
Seattle_dist     6996 non-null float64
days_end_FY      6996 non-null int64
dtypes: datetime64[ns](3), float64(4), int64(3), object(3), timedelta64[ns](1)
memory usage: 819.8+ KB


In [169]:
df.tail(2)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,days_end_FY
10998,10999,96785,9719 3rd Ave NW ...,3RD AVE NW BETWEEN NW 97TH ST AND NW 100TH ST ...,2011-12-28 08:00:00,2011-12-30 08:00:00,2011-12-30 08:00:00,2 days,2,47.701542,-122.374093,"NW 100th St, Seattle, WA 98177, USA",6.872044,-6
10999,11000,107829,Rainier Ave S to S Massachusetts ...,RAINIER AVE S BETWEEN RAINIER NB (I90 EB ON RP...,2012-03-19 07:00:00,2012-03-20 07:00:00,2012-03-20 07:00:00,1 days,1,47.588483,-122.313814,"S Massachusetts St, Seattle, WA, USA",1.492505,3


In [170]:
def get_neighborhoods(df):
    '''
    INPUT: df
    OUTPUT: df
    Pass in the cleaned data as a dataframe and add a new column
    containing the neighborhood the pothole belongs to
    '''
    # Read in shapefile of Seattle neighborhoods
    shapefilename = 'data/Neighborhoods'
    shp = fiona.open(shapefilename+'.shp')
    
    # Get neighborhood polys and their indices
    polys = [shape(pol['geometry']) for pol in shp]
    shp.close()
    neighborhood_index = [i+1 for i in range(len(polys))]

    # Associate each neighborhood polygon with its index
    neighborhoods_tup = []
    mpolys = MultiPolygon(polys)
    for hood in xrange(len(polys)):
        neighborhoods_tup.append((mpolys[hood],\
        neighborhood_index[hood]))

    # Get potholes
    with open('all_potholes.pkl') as f:
        all_potholes = pickle.load(f)
    
    # Extract the neighborhood index associated with each pothole
    neighborhood_label = []
    for hole in xrange(df.shape[0]):
        found = False
        for hood in xrange(len(neighborhoods_tup)):
            # if neighborhoods_tup[hood][0].contains(potholes_tup[hole][0]):
            if neighborhoods_tup[hood][0].contains(all_potholes[0][hole]):
                neighborhood_label.append(neighborhoods_tup[hood][1])
                found = True
        if not found:
            neighborhood_label.append('')
    
    # Add labels to dataframe
    df['neighborhood_label'] = pd.Series(neighborhood_label,\
        index = all_potholes[1])

    return df

####get_neighborhoods just crashed while indexing through the potholes [hole].  The set of potholes was re-created to match only the potholes in that new partial list of potholes.  This run needed the full list of potholes.

In [171]:
def _get_potholes(df):
    '''
    INPUT: df
    OUTPUT: None
    Create geometries for all potholes, get their indices and pickle as a tuple
    '''
    all_potholes = MultiPoint([Point(x, y) for x, y in zip(df['longitude'],\
        df['latitude'])])
    all_potholes = (all_potholes, df.index.tolist())

    with open('all_potholes.pkl', 'w') as f:
        pickle.dump(all_potholes, f)

In [172]:
_get_potholes(df)

In [173]:
df = get_neighborhoods(df)

In [174]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6996 entries, 0 to 10999
Data columns (total 15 columns):
OBJECTID              6996 non-null int64
WOKEY                 6996 non-null int64
LOCATION              6996 non-null object
ADDRDESC              6996 non-null object
INITDT_dt             6996 non-null datetime64[ns]
FLDSTARTDT_dt         6996 non-null datetime64[ns]
FLDENDDT_dt           6996 non-null datetime64[ns]
DURATION              6996 non-null timedelta64[ns]
DURATION_td           6996 non-null float64
latitude              6996 non-null float64
longitude             6996 non-null float64
address               6996 non-null object
Seattle_dist          6996 non-null float64
days_end_FY           6996 non-null int64
neighborhood_label    6996 non-null object
dtypes: datetime64[ns](3), float64(4), int64(3), object(4), timedelta64[ns](1)
memory usage: 874.5+ KB


In [175]:
def get_census_economic_vals(df):
    '''
    INPUT: df
    OUTPUT: df
    Pass in the cleaned data as a dataframe and add a new column
    containing economic values based on census data.
    '''
    # Read in shapefile of Seattle block groups
    shapefilename = 'data/tl_2013_53_bg_Seattle'
    shp = fiona.open(shapefilename+'.shp')

    # Get block group polys
    polys = [shape(pol['geometry']) for pol in shp]

    # Associate each block group polygon with its GEOID
    block_group_tup = []
    idx = 0
    mpolys = MultiPolygon(polys)
    for feature in shp:
        block_group_tup.append((mpolys[idx],\
        feature['properties']['GEOID']))
        idx += 1
    shp.close()

    # Get potholes
    with open('all_potholes.pkl') as f:
        all_potholes = pickle.load(f)

    # Extract the block group GEOID associated with each pothole
    block_group_label = []
    for hole in xrange(len(all_potholes[0])):
        found = False
        for group in xrange(len(block_group_tup)):
            if block_group_tup[group][0].contains(all_potholes[0][hole]):
                block_group_label.append(block_group_tup[group][1])
                found = True
        if not found:
            block_group_label.append('')

    # Add block group to dataframe
    df['GEOID'] = pd.Series(block_group_label,\
        index = all_potholes[1])
 
    # Read in, prep, join housing value data
#     df_econ = pd.read_csv('data/ACS_13_5YR_B25077_with_ann.csv',\
#         skiprows=range(1,2))
#     df_econ.rename(columns={'GEO.id2':'GEOID'}, inplace=True)
#     df_econ['GEOID'] = df_econ['GEOID'].astype('unicode')
#     df = pd.merge(df, df_econ, how='left', on='GEOID')

    # Convert economic values to floats, NaNs if not possible
#     df['HD01_VD01'] = df['HD01_VD01'].\
#         convert_objects(convert_numeric=True)
#     df['HD02_VD01'] = df['HD02_VD01'].\
#         convert_objects(convert_numeric=True)

    # Rename economic values
#     df.rename(columns={'HD01_VD01': 'Median_Value',\
#                        'HD02_VD01': 'Margin_of_Error'}, inplace=True)

    return df

In [176]:
df = get_census_economic_vals(df)

In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6996 entries, 0 to 10999
Data columns (total 16 columns):
OBJECTID              6996 non-null int64
WOKEY                 6996 non-null int64
LOCATION              6996 non-null object
ADDRDESC              6996 non-null object
INITDT_dt             6996 non-null datetime64[ns]
FLDSTARTDT_dt         6996 non-null datetime64[ns]
FLDENDDT_dt           6996 non-null datetime64[ns]
DURATION              6996 non-null timedelta64[ns]
DURATION_td           6996 non-null float64
latitude              6996 non-null float64
longitude             6996 non-null float64
address               6996 non-null object
Seattle_dist          6996 non-null float64
days_end_FY           6996 non-null int64
neighborhood_label    6996 non-null object
GEOID                 6996 non-null object
dtypes: datetime64[ns](3), float64(4), int64(3), object(5), timedelta64[ns](1)
memory usage: 929.2+ KB


####This is where it's re-indexed!  I bet it's the 'merge' that did it.

In [178]:
# Read in, prep, join housing value data
df_econ = pd.read_csv('data/ACS_13_5YR_B25077_with_ann.csv',\
    skiprows=range(1,2))
df_econ.rename(columns={'GEO.id2':'GEOID'}, inplace=True)
df_econ['GEOID'] = df_econ['GEOID'].astype('unicode')


In [179]:
df_econ.columns

Index([u'GEO.id', u'GEOID', u'GEO.display-label', u'HD01_VD01', u'HD02_VD01'], dtype='object')

In [180]:
df.columns

Index([u'OBJECTID', u'WOKEY', u'LOCATION', u'ADDRDESC', u'INITDT_dt',
       u'FLDSTARTDT_dt', u'FLDENDDT_dt', u'DURATION', u'DURATION_td',
       u'latitude', u'longitude', u'address', u'Seattle_dist', u'days_end_FY',
       u'neighborhood_label', u'GEOID'],
      dtype='object')

In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6996 entries, 0 to 10999
Data columns (total 16 columns):
OBJECTID              6996 non-null int64
WOKEY                 6996 non-null int64
LOCATION              6996 non-null object
ADDRDESC              6996 non-null object
INITDT_dt             6996 non-null datetime64[ns]
FLDSTARTDT_dt         6996 non-null datetime64[ns]
FLDENDDT_dt           6996 non-null datetime64[ns]
DURATION              6996 non-null timedelta64[ns]
DURATION_td           6996 non-null float64
latitude              6996 non-null float64
longitude             6996 non-null float64
address               6996 non-null object
Seattle_dist          6996 non-null float64
days_end_FY           6996 non-null int64
neighborhood_label    6996 non-null object
GEOID                 6996 non-null object
dtypes: datetime64[ns](3), float64(4), int64(3), object(5), timedelta64[ns](1)
memory usage: 929.2+ KB


In [182]:
df.tail(2)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,days_end_FY,neighborhood_label,GEOID
10998,10999,96785,9719 3rd Ave NW ...,3RD AVE NW BETWEEN NW 97TH ST AND NW 100TH ST ...,2011-12-28 08:00:00,2011-12-30 08:00:00,2011-12-30 08:00:00,2 days,2,47.701542,-122.374093,"NW 100th St, Seattle, WA 98177, USA",6.872044,-6,117,530330016001
10999,11000,107829,Rainier Ave S to S Massachusetts ...,RAINIER AVE S BETWEEN RAINIER NB (I90 EB ON RP...,2012-03-19 07:00:00,2012-03-20 07:00:00,2012-03-20 07:00:00,1 days,1,47.588483,-122.313814,"S Massachusetts St, Seattle, WA, USA",1.492505,3,79,530330094001


In [183]:
df = df.reset_index()

In [184]:
df.head(2)

,index,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,days_end_FY,neighborhood_label,GEOID
0,0,1,2456,Holden pl SW bt SW Kenyon Holden ...,HOLDEN PL SW BETWEEN SW HOLDEN ST AND SW KENYO...,2010-03-29 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,8 days,8,47.531858,-122.363186,"SW Kenyon St, Seattle, WA, USA",5.338641,3,93,530330114011
1,1,2,2514,"NE Pacific Street, one block west of Montlake ...",MONTLAKE BLVD NE AND NE PACIFIC ST ...,2010-03-30 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,2 days,2,47.649014,-122.304655,"Montlake Blvd NE & NE Pacific St, Seattle, WA ...",3.222458,3,42,530330053022


In [185]:
df.tail(2)

,index,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,days_end_FY,neighborhood_label,GEOID
6994,10998,10999,96785,9719 3rd Ave NW ...,3RD AVE NW BETWEEN NW 97TH ST AND NW 100TH ST ...,2011-12-28 08:00:00,2011-12-30 08:00:00,2011-12-30 08:00:00,2 days,2,47.701542,-122.374093,"NW 100th St, Seattle, WA 98177, USA",6.872044,-6,117,530330016001
6995,10999,11000,107829,Rainier Ave S to S Massachusetts ...,RAINIER AVE S BETWEEN RAINIER NB (I90 EB ON RP...,2012-03-19 07:00:00,2012-03-20 07:00:00,2012-03-20 07:00:00,1 days,1,47.588483,-122.313814,"S Massachusetts St, Seattle, WA, USA",1.492505,3,79,530330094001


In [189]:
df_ = pd.merge(df, df_econ, how='left', on='GEOID')

In [190]:
df_.head(2)

,index,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,days_end_FY,neighborhood_label,GEOID,GEO.id,GEO.display-label,HD01_VD01,HD02_VD01
0,0,1,2456,Holden pl SW bt SW Kenyon Holden ...,HOLDEN PL SW BETWEEN SW HOLDEN ST AND SW KENYO...,2010-03-29 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,8 days,8,47.531858,-122.363186,"SW Kenyon St, Seattle, WA, USA",5.338641,3,93,530330114011,1500000US530330114011,"Block Group 1, Census Tract 114.01, King Count...",307500,53269
1,1,2,2514,"NE Pacific Street, one block west of Montlake ...",MONTLAKE BLVD NE AND NE PACIFIC ST ...,2010-03-30 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,2 days,2,47.649014,-122.304655,"Montlake Blvd NE & NE Pacific St, Seattle, WA ...",3.222458,3,42,530330053022,1500000US530330053022,"Block Group 2, Census Tract 53.02, King County...",-,**


In [191]:
df_.tail(2)

,index,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,days_end_FY,neighborhood_label,GEOID,GEO.id,GEO.display-label,HD01_VD01,HD02_VD01
6994,10998,10999,96785,9719 3rd Ave NW ...,3RD AVE NW BETWEEN NW 97TH ST AND NW 100TH ST ...,2011-12-28 08:00:00,2011-12-30 08:00:00,2011-12-30 08:00:00,2 days,2,47.701542,-122.374093,"NW 100th St, Seattle, WA 98177, USA",6.872044,-6,117,530330016001,1500000US530330016001,"Block Group 1, Census Tract 16, King County, W...",781700,74355
6995,10999,11000,107829,Rainier Ave S to S Massachusetts ...,RAINIER AVE S BETWEEN RAINIER NB (I90 EB ON RP...,2012-03-19 07:00:00,2012-03-20 07:00:00,2012-03-20 07:00:00,1 days,1,47.588483,-122.313814,"S Massachusetts St, Seattle, WA, USA",1.492505,3,79,530330094001,1500000US530330094001,"Block Group 1, Census Tract 94, King County, W...",289200,38332


In [194]:
df_.set_index('index')

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,days_end_FY,neighborhood_label,GEOID,GEO.id,GEO.display-label,HD01_VD01,HD02_VD01
index,,,,,,,,,,,,,,,,,,,,
0,1,2456,Holden pl SW bt SW Kenyon Holden ...,HOLDEN PL SW BETWEEN SW HOLDEN ST AND SW KENYO...,2010-03-29 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,8 days 00:00:00,8,47.531858,-122.363186,"SW Kenyon St, Seattle, WA, USA",5.338641,3,93,530330114011,1500000US530330114011,"Block Group 1, Census Tract 114.01, King Count...",307500,53269
1,2,2514,"NE Pacific Street, one block west of Montlake ...",MONTLAKE BLVD NE AND NE PACIFIC ST ...,2010-03-30 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,2 days 00:00:00,2,47.649014,-122.304655,"Montlake Blvd NE & NE Pacific St, Seattle, WA ...",3.222458,3,42,530330053022,1500000US530330053022,"Block Group 2, Census Tract 53.02, King County...",-,**
2,3,2649,32nd Ave NE and NE 135th ...,NE 135TH ST BETWEEN 32ND AVE NE AND LAKE CITY ...,2010-03-31 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,1 days 00:00:00,1,47.707446,-122.301566,"Lake City Way NE, Seattle, WA, USA",7.137526,3,104,530330011001,1500000US530330011001,"Block Group 1, Census Tract 11, King County, W...",416900,62441
3,4,1591,Charles st & 6 Ave S pothole next to traffic l...,S CHARLES ST BETWEEN 6TH AVE S AND MAYNARD AVE...,2010-03-15 07:00:00,2010-03-16 07:00:00,2010-03-16 07:00:00,1 days 00:00:00,1,47.597085,-122.325065,"Maynard Ave S, Seattle, WA, USA",0.710334,3,61,530330091002,1500000US530330091002,"Block Group 2, Census Tract 91, King County, W...",328900,145688
5,6,5642,52nd St. & 18th Ave. NE ...,18TH AVE NE BETWEEN NE 52ND ST AND NE 55TH ST ...,2010-04-19 07:00:00,2010-04-22 07:00:00,2010-04-22 07:00:00,3 days 00:00:00,3,47.668522,-122.295630,"NE 55th St, Seattle, WA 98105, USA",4.628992,2,36,530330042005,1500000US530330042005,"Block Group 5, Census Tract 42, King County, W...",637000,56463
6,7,7585,...,3RD AVE NW BETWEEN NW 125TH ST AND NW 127TH ST...,2010-05-03 07:00:00,2010-05-04 07:00:00,2010-05-04 07:00:00,1 days 00:00:00,1,47.721476,-122.364984,"NW 127th St, Seattle, WA 98177, USA",8.110129,1,110,530330005002,1500000US530330005002,"Block Group 2, Census Tract 5, King County, Wa...",565200,53534
9,10,7211,Republican and Minor Ave N ...,MINOR AVE N AND REPUBLICAN ST ...,2010-04-29 07:00:00,2010-05-03 07:00:00,2010-05-03 07:00:00,4 days 00:00:00,4,47.623131,-122.333003,"Republican St & Minor Ave N, Seattle, WA 98109...",1.169861,2,49,530330073001,1500000US530330073001,"Block Group 1, Census Tract 73, King County, W...",-,**
10,11,3168,...,17TH AVE W BETWEEN W DRAVUS ST AND W BERTONA S...,2010-04-02 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,4 days 00:00:00,4,47.650311,-122.388604,"W Bertona St, Seattle, WA, USA",4.031574,2,50,530330058012,1500000US530330058012,"Block Group 2, Census Tract 58.01, King County...",255200,50905
12,13,6967,B/T Pike and Pine B/t 211 and 215 ...,PIKE ST BETWEEN 2ND AVE AND 3RD AVE ...,2010-04-28 07:00:00,2010-05-03 07:00:00,2010-05-03 07:00:00,5 days 00:00:00,5,47.610741,-122.338703,"3rd Ave, Seattle, WA, USA",0.440475,2,62,530330081002,1500000US530330081002,"Block Group 2, Census Tract 81, King County, W...",384400,224247


In [195]:
df_ = df_.set_index('index')

In [196]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6996 entries, 0 to 10999
Data columns (total 20 columns):
OBJECTID              6996 non-null int64
WOKEY                 6996 non-null int64
LOCATION              6996 non-null object
ADDRDESC              6996 non-null object
INITDT_dt             6996 non-null datetime64[ns]
FLDSTARTDT_dt         6996 non-null datetime64[ns]
FLDENDDT_dt           6996 non-null datetime64[ns]
DURATION              6996 non-null timedelta64[ns]
DURATION_td           6996 non-null float64
latitude              6996 non-null float64
longitude             6996 non-null float64
address               6996 non-null object
Seattle_dist          6996 non-null float64
days_end_FY           6996 non-null int64
neighborhood_label    6996 non-null object
GEOID                 6996 non-null object
GEO.id                6932 non-null object
GEO.display-label     6932 non-null object
HD01_VD01             6932 non-null object
HD02_VD01             6932 non-null objec

In [197]:
df_.tail(2)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,days_end_FY,neighborhood_label,GEOID,GEO.id,GEO.display-label,HD01_VD01,HD02_VD01
index,,,,,,,,,,,,,,,,,,,,
10998,10999,96785,9719 3rd Ave NW ...,3RD AVE NW BETWEEN NW 97TH ST AND NW 100TH ST ...,2011-12-28 08:00:00,2011-12-30 08:00:00,2011-12-30 08:00:00,2 days,2,47.701542,-122.374093,"NW 100th St, Seattle, WA 98177, USA",6.872044,-6,117,530330016001,1500000US530330016001,"Block Group 1, Census Tract 16, King County, W...",781700,74355
10999,11000,107829,Rainier Ave S to S Massachusetts ...,RAINIER AVE S BETWEEN RAINIER NB (I90 EB ON RP...,2012-03-19 07:00:00,2012-03-20 07:00:00,2012-03-20 07:00:00,1 days,1,47.588483,-122.313814,"S Massachusetts St, Seattle, WA, USA",1.492505,3,79,530330094001,1500000US530330094001,"Block Group 1, Census Tract 94, King County, W...",289200,38332


####OBJECTID = original index + 1

Two options: 
    1. Before merging, make the index of the left df a column, then make it the index again after merging.  
    2. Take the merged df and convert the OBJECTID column back into the original index.
In either case I have to run the closest street features run again.

####Let's do option 1.  DONE.  Now let's add code to get income and test it.

In [199]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6996 entries, 0 to 10999
Data columns (total 20 columns):
OBJECTID              6996 non-null int64
WOKEY                 6996 non-null int64
LOCATION              6996 non-null object
ADDRDESC              6996 non-null object
INITDT_dt             6996 non-null datetime64[ns]
FLDSTARTDT_dt         6996 non-null datetime64[ns]
FLDENDDT_dt           6996 non-null datetime64[ns]
DURATION              6996 non-null timedelta64[ns]
DURATION_td           6996 non-null float64
latitude              6996 non-null float64
longitude             6996 non-null float64
address               6996 non-null object
Seattle_dist          6996 non-null float64
days_end_FY           6996 non-null int64
neighborhood_label    6996 non-null object
GEOID                 6996 non-null object
GEO.id                6932 non-null object
GEO.display-label     6932 non-null object
HD01_VD01             6932 non-null object
HD02_VD01             6932 non-null objec

In [200]:
# Convert economic values to floats, NaNs if not possible
df_['HD01_VD01'] = df_['HD01_VD01'].\
    convert_objects(convert_numeric=True)
df_['HD02_VD01'] = df_['HD02_VD01'].\
    convert_objects(convert_numeric=True)


In [201]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6996 entries, 0 to 10999
Data columns (total 20 columns):
OBJECTID              6996 non-null int64
WOKEY                 6996 non-null int64
LOCATION              6996 non-null object
ADDRDESC              6996 non-null object
INITDT_dt             6996 non-null datetime64[ns]
FLDSTARTDT_dt         6996 non-null datetime64[ns]
FLDENDDT_dt           6996 non-null datetime64[ns]
DURATION              6996 non-null timedelta64[ns]
DURATION_td           6996 non-null float64
latitude              6996 non-null float64
longitude             6996 non-null float64
address               6996 non-null object
Seattle_dist          6996 non-null float64
days_end_FY           6996 non-null int64
neighborhood_label    6996 non-null object
GEOID                 6996 non-null object
GEO.id                6932 non-null object
GEO.display-label     6932 non-null object
HD01_VD01             6549 non-null float64
HD02_VD01             6549 non-null floa

In [204]:
len(df_.HD01_VD01.unique())

405

In [205]:
# Rename economic values
df_.rename(columns={'HD01_VD01': 'Median_Home_Value',\
                   'HD02_VD01': 'Home_Margin_of_Error'}, inplace=True)

In [207]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6996 entries, 0 to 10999
Data columns (total 20 columns):
OBJECTID                6996 non-null int64
WOKEY                   6996 non-null int64
LOCATION                6996 non-null object
ADDRDESC                6996 non-null object
INITDT_dt               6996 non-null datetime64[ns]
FLDSTARTDT_dt           6996 non-null datetime64[ns]
FLDENDDT_dt             6996 non-null datetime64[ns]
DURATION                6996 non-null timedelta64[ns]
DURATION_td             6996 non-null float64
latitude                6996 non-null float64
longitude               6996 non-null float64
address                 6996 non-null object
Seattle_dist            6996 non-null float64
days_end_FY             6996 non-null int64
neighborhood_label      6996 non-null object
GEOID                   6996 non-null object
GEO.id                  6932 non-null object
GEO.display-label       6932 non-null object
Median_Home_Value       6549 non-null float64
Ho

In [210]:
def _lookup_income(df, filename):
    '''
    INPUT: df, filename path to lookup table
    OUTPUT: df with income value added to dataframe
    '''
# Read in, prep, join housing value data
    df_econ = pd.read_csv(filename, skiprows=range(1,3))
    df_econ.rename(columns={'GEO.id2':'GEOID'}, inplace=True)
    df_econ['GEOID'] = df_econ['GEOID'].astype('unicode')

    df = df.reset_index()
    df = pd.merge(df, df_econ, how='left', on='GEOID')
    df = df.set_index('index')

    # Convert economic values to floats, NaNs if not possible
    df['HD01_VD01'] = df['HD01_VD01'].\
        convert_objects(convert_numeric=True)
    df['HD02_VD01'] = df['HD02_VD01'].\
        convert_objects(convert_numeric=True)

    # Rename economic values
    df.rename(columns={'HD01_VD01': 'Median_Income',\
                       'HD02_VD01': 'Income_Margin_of_Error'}, inplace=True)
    return df

In [214]:
df_ = _lookup_income(df, 'data/ACS_13_5YR_B19013_with_ann.csv')

In [215]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6996 entries, 0 to 10999
Data columns (total 25 columns):
level_0                   6996 non-null int64
OBJECTID                  6996 non-null int64
WOKEY                     6996 non-null int64
LOCATION                  6996 non-null object
ADDRDESC                  6996 non-null object
INITDT_dt                 6996 non-null datetime64[ns]
FLDSTARTDT_dt             6996 non-null datetime64[ns]
FLDENDDT_dt               6996 non-null datetime64[ns]
DURATION                  6996 non-null timedelta64[ns]
DURATION_td               6996 non-null float64
latitude                  6996 non-null float64
longitude                 6996 non-null float64
address                   6996 non-null object
Seattle_dist              6996 non-null float64
days_end_FY               6996 non-null int64
neighborhood_label        6996 non-null object
GEOID                     6996 non-null object
GEO.id_x                  6932 non-null object
GEO.display-l

In [216]:
df_.head(2)

,level_0,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,days_end_FY,neighborhood_label,GEOID,GEO.id_x,GEO.display-label_x,Median_Income,Income_Margin_of_Error,GEO.id_y,GEO.display-label_y,Median_Income,Income_Margin_of_Error
index,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,1,2456,Holden pl SW bt SW Kenyon Holden ...,HOLDEN PL SW BETWEEN SW HOLDEN ST AND SW KENYO...,2010-03-29 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,8 days,8,47.531858,-122.363186,"SW Kenyon St, Seattle, WA, USA",5.338641,3,93,530330114011,1500000US530330114011,"Block Group 1, Census Tract 114.01, King Count...",53523,15208,1500000US530330114011,"Block Group 1, Census Tract 114.01, King Count...",53523,15208
1,1,2,2514,"NE Pacific Street, one block west of Montlake ...",MONTLAKE BLVD NE AND NE PACIFIC ST ...,2010-03-30 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,2 days,2,47.649014,-122.304655,"Montlake Blvd NE & NE Pacific St, Seattle, WA ...",3.222458,3,42,530330053022,1500000US530330053022,"Block Group 2, Census Tract 53.02, King County...",NaN,NaN,1500000US530330053022,"Block Group 2, Census Tract 53.02, King County...",NaN,NaN


In [217]:
df_.tail(2)

,level_0,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,days_end_FY,neighborhood_label,GEOID,GEO.id_x,GEO.display-label_x,Median_Income,Income_Margin_of_Error,GEO.id_y,GEO.display-label_y,Median_Income,Income_Margin_of_Error
index,,,,,,,,,,,,,,,,,,,,,,,,,
10998,6994,10999,96785,9719 3rd Ave NW ...,3RD AVE NW BETWEEN NW 97TH ST AND NW 100TH ST ...,2011-12-28 08:00:00,2011-12-30 08:00:00,2011-12-30 08:00:00,2 days,2,47.701542,-122.374093,"NW 100th St, Seattle, WA 98177, USA",6.872044,-6,117,530330016001,1500000US530330016001,"Block Group 1, Census Tract 16, King County, W...",88854,15902,1500000US530330016001,"Block Group 1, Census Tract 16, King County, W...",88854,15902
10999,6995,11000,107829,Rainier Ave S to S Massachusetts ...,RAINIER AVE S BETWEEN RAINIER NB (I90 EB ON RP...,2012-03-19 07:00:00,2012-03-20 07:00:00,2012-03-20 07:00:00,1 days,1,47.588483,-122.313814,"S Massachusetts St, Seattle, WA, USA",1.492505,3,79,530330094001,1500000US530330094001,"Block Group 1, Census Tract 94, King County, W...",29440,40703,1500000US530330094001,"Block Group 1, Census Tract 94, King County, W...",29440,40703


In [103]:
df[df.neighborhood_label == ''].shape, df[df.GEOID == ''].shape

((237, 20), (64, 20))

In [106]:
df[pd.isnull(df.Median_Value)].shape

(447, 20)

####OK, I know why I'm not gettting any street features in my run.  My indexing was re-set by my merge operation, so that the indexes did not match, leading to zeros/NaNs for all my street features.

In [219]:
def _lookup_housing(df, filename):
    '''
    INPUT: df, filename path to lookup table
    OUTPUT: df with econ value added to dataframe
    '''
    # Read in, prep, join housing value data
    df_econ = pd.read_csv(filename, skiprows=range(1,2))
    df_econ.rename(columns={'GEO.id2':'GEOID'}, inplace=True)
    df_econ['GEOID'] = df_econ['GEOID'].astype('unicode')

    df = df.reset_index()
    df = pd.merge(df, df_econ, how='left', on='GEOID')
    df = df.set_index('index')

    # Convert economic values to floats, NaNs if not possible
    df['HD01_VD01'] = df['HD01_VD01'].\
        convert_objects(convert_numeric=True)
    df['HD02_VD01'] = df['HD02_VD01'].\
        convert_objects(convert_numeric=True)

    # Rename economic values
    df.rename(columns={'HD01_VD01': 'Median_Home_Value',\
                       'HD02_VD01': 'Home_Margin_of_Error'}, inplace=True)

    return df

In [220]:
def get_census_economic_vals(df):
    '''
    INPUT: df
    OUTPUT: df
    Pass in the cleaned data as a dataframe and add a new column
    containing economic values based on census data.
    '''
    # Read in shapefile of Seattle block groups
    shapefilename = 'data/tl_2013_53_bg_Seattle'
    shp = fiona.open(shapefilename+'.shp')

    # Get block group polys
    polys = [shape(pol['geometry']) for pol in shp]

    # Associate each block group polygon with its GEOID
    block_group_tup = []
    idx = 0
    mpolys = MultiPolygon(polys)
    for feature in shp:
        block_group_tup.append((mpolys[idx],\
        feature['properties']['GEOID']))
        idx += 1
    shp.close()

    # Get potholes
    with open('all_potholes.pkl') as f:
        all_potholes = pickle.load(f)

    # Extract the block group GEOID associated with each pothole
    block_group_label = []
    for hole in xrange(len(all_potholes[0])):
        found = False
        for group in xrange(len(block_group_tup)):
            if block_group_tup[group][0].contains(all_potholes[0][hole]):
                block_group_label.append(block_group_tup[group][1])
                found = True
        if not found:
            block_group_label.append('')

    # Add block group to dataframe
    df['GEOID'] = pd.Series(block_group_label,\
        index = all_potholes[1])
 
    df = _lookup_housing(df, 'data/ACS_13_5YR_B25077_with_ann.csv')
    df = _lookup_income(df, 'data/ACS_13_5YR_B19013_with_ann.csv')

    return df


In [218]:
df = pd.read_pickle('df_1to10999_geo_cleaned.pkl')

In [221]:
df = get_census_economic_vals(df)

In [222]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6996 entries, 0 to 10999
Data columns (total 21 columns):
OBJECTID                  6996 non-null int64
WOKEY                     6996 non-null int64
LOCATION                  6996 non-null object
ADDRDESC                  6996 non-null object
INITDT_dt                 6996 non-null datetime64[ns]
FLDSTARTDT_dt             6996 non-null datetime64[ns]
FLDENDDT_dt               6996 non-null datetime64[ns]
DURATION                  6996 non-null timedelta64[ns]
DURATION_td               6996 non-null float64
latitude                  6996 non-null float64
longitude                 6996 non-null float64
address                   6996 non-null object
GEOID                     6996 non-null object
GEO.id_x                  6932 non-null object
GEO.display-label_x       6932 non-null object
Median_Home_Value         6549 non-null float64
Home_Margin_of_Error      6549 non-null float64
GEO.id_y                  6932 non-null object
GEO.displa

In [223]:
df.head(2)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,GEOID,GEO.id_x,GEO.display-label_x,Median_Home_Value,Home_Margin_of_Error,GEO.id_y,GEO.display-label_y,Median_Income,Income_Margin_of_Error
index,,,,,,,,,,,,,,,,,,,,,
0,1,2456,Holden pl SW bt SW Kenyon Holden ...,HOLDEN PL SW BETWEEN SW HOLDEN ST AND SW KENYO...,2010-03-29 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,8 days,8,47.531858,-122.363186,"SW Kenyon St, Seattle, WA, USA",530330114011,1500000US530330114011,"Block Group 1, Census Tract 114.01, King Count...",307500,53269,1500000US530330114011,"Block Group 1, Census Tract 114.01, King Count...",53523,15208
1,2,2514,"NE Pacific Street, one block west of Montlake ...",MONTLAKE BLVD NE AND NE PACIFIC ST ...,2010-03-30 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,2 days,2,47.649014,-122.304655,"Montlake Blvd NE & NE Pacific St, Seattle, WA ...",530330053022,1500000US530330053022,"Block Group 2, Census Tract 53.02, King County...",NaN,NaN,1500000US530330053022,"Block Group 2, Census Tract 53.02, King County...",NaN,NaN


In [224]:
df.tail(2)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,GEOID,GEO.id_x,GEO.display-label_x,Median_Home_Value,Home_Margin_of_Error,GEO.id_y,GEO.display-label_y,Median_Income,Income_Margin_of_Error
index,,,,,,,,,,,,,,,,,,,,,
10998,10999,96785,9719 3rd Ave NW ...,3RD AVE NW BETWEEN NW 97TH ST AND NW 100TH ST ...,2011-12-28 08:00:00,2011-12-30 08:00:00,2011-12-30 08:00:00,2 days,2,47.701542,-122.374093,"NW 100th St, Seattle, WA 98177, USA",530330016001,1500000US530330016001,"Block Group 1, Census Tract 16, King County, W...",781700,74355,1500000US530330016001,"Block Group 1, Census Tract 16, King County, W...",88854,15902
10999,11000,107829,Rainier Ave S to S Massachusetts ...,RAINIER AVE S BETWEEN RAINIER NB (I90 EB ON RP...,2012-03-19 07:00:00,2012-03-20 07:00:00,2012-03-20 07:00:00,1 days,1,47.588483,-122.313814,"S Massachusetts St, Seattle, WA, USA",530330094001,1500000US530330094001,"Block Group 1, Census Tract 94, King County, W...",289200,38332,1500000US530330094001,"Block Group 1, Census Tract 94, King County, W...",29440,40703


####OK, this works.  And, oh, snap-diddly, I now know why I'm getting so many NaNs in the street values.  It's the same reason.  Even for the earlier data.  I reindexed when I merged, so the new indices, which started from 0 straight through did not match the original indices from the potholes.  Those street features are potentially mostly available!!!  But I have to rerun the entire datasest, which could take 8 hours or so.  Would have to do overnight.